In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

class DataAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        
    #determine if excel file matches required format    
    def is_valid_file(self):
        """Returns TRue if excel file matches required format, otherwise False"""
            
        # CH: the only thing that can create an exception is pd.ExcelFile(), 
        # so here I'm bailing out on exception, otherwise do the other checks
        try:
            # Read the Excel file and check if the required sheets exist
            excel_data = pd.ExcelFile(self.file_path)
        except Exception as e:
            print(f"Error: {e}")
            return False

        required_sheets = ['Scalars', 'Sim Raw EEG']  # Modify as per your requirements
        for sheet in required_sheets:
            if sheet not in excel_data.sheet_names:
                return False
        
        # Add more checks, such as column names, data range, etc.            
        return True

    
    def read_scalars(self, sheet_name_scalar):
        # Read the Excel file and extract the desired sheet into a DataFrame
        df = pd.read_excel(self.file_path, sheet_name=sheet_name_scalar)
        values_dict = {}
        
        # Iterate through the DataFrame and extract mean frequencies for each electrode site
        for index, row in df.iterrows():
            if row['Channel'] != '' and row['Value'].startswith('MEAN.'):
                key = df.iloc[index, 0]  # Access the first column for channel name
                value = df.iloc[index, 2]  # Access the second column for the value
                values_dict[key] = value
        
        return values_dict
    
    def plot_mean_frequency(self, values_dict):
        # Extract channel locations (x) and mean frequencies (y) from the dictionary
        x = list(values_dict.keys())
        y = list(values_dict.values())

        # Plot a scatter plot of mean frequencies
        plt.scatter(x, y)
        plt.xlabel('Channel Location')
        plt.ylabel('Frequency')
        plt.title('Mean Frequency')
        plt.xticks(rotation=90)
        plt.show()
    
    def find_extreme_numbers(self, sheet_name='Sim Raw EEG', start_row=89, num_rows=21, num_cols=21):
        # Read the Excel file and extract the desired table for pure coherence
        df = pd.read_excel(self.file_path, sheet_name=sheet_name, header=None, skiprows=start_row-1, nrows=num_rows, usecols=range(num_cols))
        df.columns = df.iloc[0]  # Use the first row as column names
        df = df.iloc[1:]  # Skip the first row since it is now used as column names
        df.set_index(df.columns[0], inplace=True)  # Set the first column as row index
        df = df.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

        # Find the three lowest and three highest values
        min_values = df.unstack().dropna().nsmallest(3)
        max_values = df.unstack().dropna().nlargest(3)

        # Store the lowest values and their corresponding locations in a dictionary
        lowest_dict = {}
        for idx, value in min_values.iteritems():
            row_header = idx[1]
            col_header = idx[0]
            lowest_dict[(row_header, col_header)] = value

        # Store the highest values and their corresponding locations in a dictionary
        highest_dict = {}
        for idx, value in max_values.iteritems():
            row_header = idx[1]
            col_header = idx[0]
            highest_dict[(row_header, col_header)] = value

        return lowest_dict, highest_dict
    
    def plot_extreme_numbers(self, lowest_dict, highest_dict):
        # Extract the keys (column headers) and values from lowest_dict
        lowest_keys = [str(key) for key in lowest_dict.keys()]
        lowest_values = list(lowest_dict.values())

        # Extract the keys (column headers) and values from highest_dict
        highest_keys = [str(key) for key in highest_dict.keys()]
        highest_values = list(highest_dict.values())

        # Plot a bar graph of the lowest and highest values
        plt.bar(lowest_keys, lowest_values, label='Lowest')
        plt.bar(highest_keys, highest_values, label='Highest')

        plt.xlabel('Columns')
        plt.ylabel('Values')
        plt.title('Lowest and Highest Values')
        plt.legend()
        plt.xticks(rotation=90)
        plt.show()




In [ ]:


# TO DO:
# determine if the file is a match
#get file name/locaiton
#file_name = input("Input the file name")
#print(file_name)
'''
match = False

#TO DO: ensure that the file inputted matches necessary format -- could also put this in the class
def file_match():
    while match == False:
        # determine what makes a "match":
        1) Is an excel spreadsheet
        2) Contains the necessary pages - "Scalars", "coherence", etc.
        --> could use file_name.info() 
        return None
    
    return None
    '''

# TO DO:
#import excel file - will have to adjust this to work for any inputted file
#data_file = pd.read_excel('Analysis_EO_2022.10.17_13.00.34.xlsx', sheet_name = 'Scalars')
#display(data_file)

 

In [9]:
# https://github.com/crahan/ipyfilechooser

# to install: pip3 install ipyfilechooser
from ipyfilechooser import FileChooser

# Note that this has to be done in a separate cell b/c it won't wait for user input otherwise
fc = FileChooser('.', filter_pattern=['*.xlsx', '*.xls'], title = '<b>Please Select data file</b>')
display(fc)
file_path = fc.selected

FileChooser(path='C:\Users\charding\Box\HCI584\student_code\EEG-Analysis-Program-HCI584', filename='', title='…

In [ ]:
# This cell set's up the analyser and validates it.  
sheet_name_scalar = 'Scalars' # make this a attribute within init
analyzer = DataAnalyzer(file_path)
analyzer.validate_file_path()  # Returns a actionable error message or something like "Valid file, proceed to next cell"

In [ ]:
# In this cell the user creates a specific plot
# Note that inside this method, you need to read in the data and analyse it like you're doing below
# This means the user just has to run this one cell to get the effect w/o having to deal with the details.
# if outfile is not None, the plot will also be saved into this file, with the given size
analyzer.extreme_numbers_plot(outfile=None, pagesize=(11, 8.5))

In [ ]:
def main():
    #file_path = '/Users/kaleighcarlone/Downloads/Analysis_EO_2022.10.17_13.00.34.xlsx'
    file_path = './Analysis_EO_2022.10.17_13.00.34.xlsx'
    sheet_name_scalar = 'Scalars'
    analyzer = DataAnalyzer(file_path)
    
    #determine if excel file is valid, if so, perform analysis.
    if analyzer.is_valid_file():
        values_dict = analyzer.read_scalars(sheet_name_scalar)
        print(values_dict)

        lowest_dict, highest_dict = analyzer.find_extreme_numbers()
        print(lowest_dict)
        print(highest_dict)

        analyzer.plot_extreme_numbers(lowest_dict, highest_dict)
        analyzer.plot_mean_frequency(values_dict)
    else:
        print("Invalid Excel file. Please check the file format and contents.")

if __name__ == '__main__':
    main()